In [1]:
import pandas as pd
df50_index= pd.read_excel(r"中证500指数（CCt）.xlsx", engine='openpyxl')
df50_index.head()


,指标名称,中证500指数
0,频率,日
1,单位,点
2,指标ID,M002845725
3,2022-12-30,5864.47
4,2022-12-29,5857.4


In [2]:
#drop first 3 row
df50_index = df50_index.drop([0,1,2])
#change column name
df50_index.columns=['date', 'index_level']

In [3]:
df50_index['date']=pd.to_datetime(df50_index['date'])
df50_index.set_index("date", inplace=True)
weekly_groups=df50_index.groupby(pd.Grouper(freq='W'))
#find the last day of each week
df_index_weekly=weekly_groups.last()
df_index_weekly
df_index_weekly.isnull().sum()
df_index_weekly.dropna(inplace=True)

In [4]:

#remove first row
df_index_weekly = df_index_weekly.iloc[1:]
df_index_weekly

,index_level
date,
2013-04-21,3495.24
2013-04-28,3367.77
2013-05-05,3448.64
2013-05-12,3577.33
2013-05-19,3691.40
...,...
2022-12-04,6168.01
2022-12-11,6192.30
2022-12-18,6072.86


In [5]:
#add index week1 week2 week3...
import numpy as np
df_index_weekly['week'] = range(1, len(df_index_weekly) + 1)
df_index_weekly.set_index("week", inplace=True)
df_index_weekly


,index_level
week,
1,3495.24
2,3367.77
3,3448.64
4,3577.33
5,3691.40
...,...
495,6168.01
496,6192.30
497,6072.86


In [6]:
#last 52 weeks' max value for weekly_indexLevel Dataset
df_index_weekly['max_52'] = None
for i in range(52, len(df_index_weekly)+1):
    previous_52 = df_index_weekly.loc[i-52:i-1, 'index_level']  
    max_value = previous_52.max()  
    df_index_weekly.at[i, 'max_52'] = max_value  

df_index_weekly

,index_level,max_52
week,,
1,3495.24,None
2,3367.77,None
3,3448.64,None
4,3577.33,None
5,3691.40,None
...,...,...
495,6168.01,7359.4
496,6192.30,7359.4
497,6072.86,7359.4


In [7]:
#calculate ML = index_level/(max of last 52 weeks) and create a new column
df_index_weekly['ML'] = df_index_weekly['index_level']/df_index_weekly['max_52']
df_index_weekly

,index_level,max_52,ML
week,,,
1,3495.24,None,NaN
2,3367.77,None,NaN
3,3448.64,None,NaN
4,3577.33,None,NaN
5,3691.40,None,NaN
...,...,...,...
495,6168.01,7359.4,0.838113
496,6192.30,7359.4,0.841414
497,6072.86,7359.4,0.825184


In [8]:
#find the mean and standard deviation of ML for past 52 weeks
df_index_weekly['mean_52'] = None
df_index_weekly['std_52'] = None
for i in range(104, len(df_index_weekly)+1):
    previous_52 = df_index_weekly.loc[i-52:i-1, 'ML']  
    mean_value = previous_52.mean()  
    std_value = previous_52.std()  
    df_index_weekly.at[i, 'mean_52'] = mean_value  
    df_index_weekly.at[i, 'std_52'] = std_value

df_index_weekly

,index_level,max_52,ML,mean_52,std_52
week,,,,,
1,3495.24,None,NaN,None,None
2,3367.77,None,NaN,None,None
3,3448.64,None,NaN,None,None
4,3577.33,None,NaN,None,None
5,3691.40,None,NaN,None,None
...,...,...,...,...,...
495,6168.01,7359.4,0.838113,0.845119,0.062085
496,6192.30,7359.4,0.841414,0.843112,0.060544
497,6072.86,7359.4,0.825184,0.841043,0.058663


In [9]:
df_index_weekly['mean-1.5*std'] = df_index_weekly['mean_52'] - 1.5*df_index_weekly['std_52']
df_index_weekly['CC'] = np.where(df_index_weekly['ML'] < df_index_weekly['mean-1.5*std'], 1, 0)
df_index_weekly

,index_level,max_52,ML,mean_52,std_52,mean-1.5*std,CC
week,,,,,,,
1,3495.24,None,NaN,None,None,NaN,0
2,3367.77,None,NaN,None,None,NaN,0
3,3448.64,None,NaN,None,None,NaN,0
4,3577.33,None,NaN,None,None,NaN,0
5,3691.40,None,NaN,None,None,NaN,0
...,...,...,...,...,...,...,...
495,6168.01,7359.4,0.838113,0.845119,0.062085,0.751991,0
496,6192.30,7359.4,0.841414,0.843112,0.060544,0.752296,0
497,6072.86,7359.4,0.825184,0.841043,0.058663,0.753048,0


In [10]:
#remove first 104 rows
df_index_weekly = df_index_weekly.iloc[104:]
df_index_weekly

,index_level,max_52,ML,mean_52,std_52,mean-1.5*std,CC
week,,,,,,,
105,7980.76,7952.19,1.003593,0.991328,0.043609,0.925915,0
106,8419.10,7980.76,1.054925,0.991885,0.043576,0.926521,0
107,8469.67,8419.1,1.006007,0.994493,0.043205,0.929685,0
108,8201.29,8469.67,0.968313,0.99632,0.041583,0.933946,0
109,8742.89,8469.67,1.032259,0.997656,0.039452,0.938478,0
...,...,...,...,...,...,...,...
495,6168.01,7359.4,0.838113,0.845119,0.062085,0.751991,0
496,6192.30,7359.4,0.841414,0.843112,0.060544,0.752296,0
497,6072.86,7359.4,0.825184,0.841043,0.058663,0.753048,0


In [11]:
#to csv
df_index_weekly.to_csv(r'500_IL_Weekly_ML.csv', index = True, header=True)

In [12]:
df_index_weekly.describe()

,index_level,CC
count,395.000000,395.000000
mean,6147.324203,0.167089
std,979.473305,0.373528
min,4127.990000,0.000000
25%,5588.455000,0.000000
50%,6209.430000,0.000000
75%,6526.155000,0.000000
max,11545.890000,1.000000
